In [17]:
import os
import asyncio
import operator
import requests
import warnings
from typing import TypedDict, Annotated, List, Union
warnings.filterwarnings("ignore", category=DeprecationWarning)

from pydantic import BaseModel, Field
from IPython.display import display, Markdown
from pprint import pprint
from dotenv import load_dotenv

from langchain_groq import ChatGroq
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.tools import TavilySearchResults

In [18]:
load_dotenv()

True

In [19]:
api_key = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Autonomous Agent"

user_key = os.getenv("PUSHOVER_KEY")
api_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [20]:
#  CONFIGURATION & INSTRUCTIONS 
HOW_MANY_SEARCHES = 5

PLANNER_INSTRUCTIONS = "You are a helpful research assistant. Given a query, come up with a set of web searches to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for. Focus on recent and distinct queries."

SEARCH_INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and produce a concise, factual summary of the results. The summary must be 2-3 paragraphs. Capture the main points, especially specific entity names, dates, and version numbers. Write clearly. Do not include any additional commentary other than the summary itself."

WRITER_INSTRUCTIONS = (
    "You are a senior researcher writing a comprehensive, in-depth professional report.\n\n"
    "CRITICAL REQUIREMENTS:\n"
    "1. LENGTH: Your report MUST be between 1500-2000 words. This is not optional.\n"
    "2. DEPTH: Provide extensive detail, multiple examples, comparisons, and analysis for each topic.\n"
    "3. STRUCTURE: Use clear markdown sections with proper headings (##, ###).\n"
    "4. ACCURACY: ONLY use information from the provided search results. Do NOT invent frameworks or data.\n"
    "5. SPECIFICITY: Include specific names, versions, dates, market data, and technical details found in search results.\n"
    "6. COMPARISONS: Provide detailed comparisons between frameworks, highlighting strengths/weaknesses.\n"
    "7. EXAMPLES: Include concrete use cases and implementation examples where available.\n\n"
    "If search data is missing on a topic, explicitly state: 'I could not find information on [topic]'.\n"
    "DO NOT mention 'TensorFlow Agents' or 'PyTorch Agents' unless they appear in the actual search results.\n\n"
    "Remember: The report must be COMPREHENSIVE (1500-2000 words) with substantial depth and detail."
)

PUSH_INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""

#### SCHEMA
##### This is How the Output will be Arranged

In [21]:
class WebSearchItem(BaseModel):
    reason: str = Field(description = " Your reasoning for why this search is import to the query")
    query: str = Field(descrition = "The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: List[WebSearchItem] = Field(description = "A list of web searches performed.")

class ReportData(BaseModel):
    short_summary: str = Field(description = "A short 2-3 sentence summary of the findings.")
    markdown_report: str = Field(descrition = "The final report.")
    follow_up_question: str = Field(description = "Suggested topics to research further.")

#### TOOLS

In [26]:
@tool
def web_search_tool(query: str) -> str:
    """Search the web for the given term. Use this for research."""
    try:
        search = TavilySearchResults(max_results = 3)
        return str(search.invoke(query))
    except Exception as e:
        return f"Error performing search :{e}"

@tool
def push_notification_tool(message: str):
    """Send a push notification with this brief message"""
    if not user_key or not api_token:
        return "Error: PUSHOVER USER OR PUSHOVER TOKEN not found in the Environment"
    payload = {"user": user_key, "token": api_token, "message": message }
    pushover_url = "https://api.pushover.net/1/messages.json"

    try:
        response = requests.post(pushover_url, data= payload)
        if response.status_code == 200:
            return "Success"
        else:
            return f"Failed to send notification: {response.text}"
    except Exception as e:
        return f"Error sending notification: {e}"
